In [1]:
import torch
from torch import nn
from matplotlib import pyplot as plt 
import numpy as np
import wandb
import time
from sklearn.model_selection import train_test_split


Data Preprocessing

In [25]:
out = np.load('training-val-test-data.npz')
th_train = out['th'] #th[0],th[1],th[2],th[3],...
u_train = out['u'] #u[0],u[1],u[2],u[3],...

# data = np.load('test-prediction-submission-file.npz')
data = np.load('hidden-test-prediction-submission-file.npz')
upast_test = data['upast'] #N by u[k-15],u[k-14],...,u[k-1]
thpast_test = data['thpast'] #N by y[k-15],y[k-14],...,y[k-1]
# thpred = data['thnow'] #all zeros


def create_IO_data(u,y,na,nb):
    X = []
    Y = []
    for k in range(max(na,nb), len(y)):
        X.append(np.concatenate([u[k-nb:k],y[k-na:k]]))
        Y.append(y[k])
    return np.array(X), np.array(Y)

na = 2
nb = 3
Xtrain, Ytrain = create_IO_data(u_train, th_train, na, nb)

In [26]:
class Network(nn.Module):
    def __init__(self, n_hidden_nodes=40, n_in=nb+na):
        super(Network,self).__init__()
        self.layer1 = nn.Linear(n_in,n_hidden_nodes).double()
        self.layer2 = nn.Linear(n_hidden_nodes,n_hidden_nodes).double()
        self.layer3 = nn.Linear(n_hidden_nodes,1).double()

    def forward(self,u):
        # u = u[:, None]
        x1 = torch.nn.functional.leaky_relu(self.layer1(u))
        x2 = torch.nn.functional.leaky_relu(self.layer2(x1))
        y = self.layer3(x2)[:, 0]
        return y


In [27]:
model = Network()
model.load_state_dict(torch.load("model.pt", weights_only=True))
model.eval()

RuntimeError: Error(s) in loading state_dict for Network:
	size mismatch for layer1.weight: copying a param with shape torch.Size([40, 20]) from checkpoint, the shape in current model is torch.Size([40, 4]).

In [15]:
def simulate(model, upast, thpast, na=10, nb=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model.eval()
    upast = upast.copy()
    thpast = thpast.copy()
    N = upast.shape[0]
    y_pred = []

    for i in range(N):
        u_seq = list(upast[i, -nb:])
        y_seq = list(thpast[i, -na:])
        preds = []

        for _ in range(50):  # or however long you want to simulate
            inp = np.concatenate([u_seq[-nb:], y_seq[-na:]])
            inp_tensor = torch.tensor(inp).double().unsqueeze(0).to(device)
            with torch.no_grad():
                y_new = model(inp_tensor).cpu().numpy()[0]
            preds.append(y_new)
            y_seq.append(y_new)
            y_seq = y_seq[-na:]

        y_pred.append(preds)

    return np.array(y_pred)

model = Network()
model.load_state_dict(torch.load("model.pt", weights_only=True))

data = np.load('hidden-test-simulation-submission-file.npz')
u_train = data['u'] 
th_train = data['th'] #only the first 50 values are filled the rest are zeros

def create_IO_data(u,y,na,nb):
    X = []
    Y = []
    for k in range(max(na,nb), len(y)):
        X.append(np.concatenate([u[k-nb:k],y[k-na:k]]))
        Y.append(y[k])
    return np.array(X), np.array(Y)

na = 2
nb = 3
Xtrain, Ytrain = create_IO_data(u_train, th_train, na, nb)

data = np.load('hidden-test-simulation-submission-file.npz')
u_test = data['u']
th_test = data['th'] #only the first 50 values are filled the rest are zeros


In [16]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(Xtrain,Ytrain)

Ytrain_pred = reg.predict(Xtrain)
print('train prediction errors:')
print('RMS:', np.mean((Ytrain_pred-Ytrain)**2)**0.5,'radians')
print('RMS:', np.mean((Ytrain_pred-Ytrain)**2)**0.5/(2*np.pi)*360,'degrees')
print('NRMS:', np.mean((Ytrain_pred-Ytrain)**2)**0.5/Ytrain.std()*100,'%')


train prediction errors:
RMS: 0.006480454031031707 radians
RMS: 0.37130266530665823 degrees
NRMS: 19.732015645890392 %


In [17]:
def simulation_IO_model(f, ulist, ylist, skip=50):

    upast = ulist[skip-na:skip].tolist() #good initialization
    ypast = ylist[skip-nb:skip].tolist()
    Y = ylist[:skip].tolist()
    for u in ulist[skip:]:
        x = np.concatenate([upast,ypast],axis=0)
        ypred = f(x)
        Y.append(ypred)
        upast.append(u)
        upast.pop(0)
        ypast.append(ypred)
        ypast.pop(0)
    return np.array(Y)

skip = max(na,nb)
th_train_sim = simulation_IO_model(lambda x: reg.predict(x[None,:])[0], u_train, th_train, skip=skip)
print('train simulation errors:')
print('RMS:', np.mean((th_train_sim[skip:]-th_train[skip:])**2)**0.5,'radians')
print('RMS:', np.mean((th_train_sim[skip:]-th_train[skip:])**2)**0.5/(2*np.pi)*360,'degrees')
print('NRMS:', np.mean((th_train_sim[skip:]-th_train[skip:])**2)**0.5/th_train.std()*100,'%')


skip = 50
th_test_sim = simulation_IO_model(lambda x: reg.predict(x[None,:])[0], u_test, th_test, skip=skip)

assert len(th_test_sim)==len(th_test)

train simulation errors:
RMS: 0.047552863640730816 radians
RMS: 2.7245783903749823 degrees
NRMS: 144.45323189933524 %


In [24]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def nn_predict(x_np):
    x_tensor = torch.tensor(x_np, dtype=torch.float64).unsqueeze(0).to("cpu")  # shape (1, 2*na)
    with torch.no_grad():
        y_pred = model(x_tensor).cpu().numpy()[0]
    return y_pred

skip = max(na, nb)
th_train_sim = simulation_IO_model(nn_predict, u_train, th_train, skip=skip)

print('train simulation errors:')
print('RMS:', np.mean((th_train_sim[skip:] - th_train[skip:])**2)**0.5, 'radians')
print('RMS:', np.mean((th_train_sim[skip:] - th_train[skip:])**2)**0.5 / (2 * np.pi) * 360, 'degrees')
print('NRMS:', np.mean((th_train_sim[skip:] - th_train[skip:])**2)**0.5 / th_train.std() * 100, '%')

skip = 50
th_test_sim = simulation_IO_model(lambda x: model(x[None,:])[0], u_test, th_test, skip=skip)

assert len(th_test_sim)==len(th_test)

Using device: cuda


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x5 and 20x40)